In [ ]:
import h5py
import tarfile
from PIL import  Image
import os
import numpy as np
import time
import pandas as pd 

In [ ]:
def write_hdf5(csv_in, root_dir, data_dir ,out_hdf5_file_name):
    dataframe_train = pd.read_csv(csv_in)
    for tv in [data_dir]:
        target_dir = os.path.join(root_dir, tv) 

        target_files = list(dataframe_train['Image_Name'])
        encoding = list(dataframe_train['encoding'])

        perm = np.random.permutation(len(dataframe_train['encoding']))
        encoding = np.array(encoding)
        encoding = encoding[perm]

        target_files = np.array(target_files)
        target_files = target_files[perm]

        img_file = list()
        for file in target_files:
            img_file.append(os.path.join(target_dir,file))
        data_files_list = list()
        for file in img_file:
            temp = Image.open(file)
            keep=temp.copy()
            data_files_list.append(keep)
            temp.close()
    print("creating file")
    fi = h5py.File(out_hdf5_file_name, 'w')
    print("creating dset")
    dset = fi.create_dataset('data', shape=(len(encoding), 256, 256))
    print("creating lset")
    lset = fi.create_dataset('labels', shape=(len(encoding),1), dtype=int)
    startt = time.time()
    for idx, (f, l) in enumerate(zip(data_files_list, encoding)):
        dset[idx] = f
        lset[idx] = l
        now = time.time()
        time_remaining = len(data_files_list) * (now - startt) / (idx + 1)
        if idx % 10 == 0:
            print(idx, time_remaining / 60, f, l)
    fi.close()

In [ ]:
csv_in = "D:\TUD\TU_Dresden\WiSe_2021\Thesis_FZJ/tbc_with_lifetime/pretraining_trainset.csv"
root_dir = "D:/TUD/TU_Dresden/WiSe_2021/Thesis_FZJ/tbc_with_lifetime/"
data_dir = "data_with_Lifetime"
out_hdf5_file_name = "tbc_train_data_hdf5.h5"

In [ ]:
write_hdf5(csv_in, root_dir, data_dir, out_hdf5_file_name)

In [ ]:
csv_in = "D:\TUD\TU_Dresden\WiSe_2021\Thesis_FZJ/tbc_with_lifetime/fine_tune_validation.csv"
out_hdf5_file_name = "tbc_validation_data_hdf5.h5"

In [ ]:
write_hdf5(csv_in, root_dir, data_dir, out_hdf5_file_name)